In [5]:

#useful: https://www.kdnuggets.com/2018/04/robust-word2vec-models-gensim.html

import csv
import nltk
import re
import numpy as np
import datetime
import time
import gensim, logging
import gensim
import gensim.corpora as corpora
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist
from scipy.spatial.distance import cosine
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from scipy.spatial import cKDTree
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import spacy
from scipy import sparse
import nltk; nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend([u'like',u'ebay', u'message', u'seller', u'sell', u'see', u'website', u'site', u'new', u'star', u'customer', u'wish', u'paypal', u'ca', u'page', u'android', u'shop', u'way', u'ad', u'let', u'add', u'team', u'able',u'library', u'hear', u'thing', u'really', u'continue', u'job', u'everyday', u'basic', u'choice', u'lil', u'month', u'save', u'come', u'ever', u'guy', u'keep', u'wifi', u'sometimes', u'artist', u'selection', u'choose', u'always', u'nothing', u'tablet', u'range', u'know', u'follow', u'daily', u'overall', u'give', u'totally', u'life', u'anything', u'everything', u'never', u'spotify', u'think', u'also', u'many', u'stuff', u'second', u'could', u'try', u'still', u'must', u'need', u'away', u'day', u'bit', u'number', u'make',u'year', u'lot', u'time',u'buy', u'song', u'even', u'minute', u'phone', u'gemitin', u'basta', u'say', u'everyone', u'service', u'take', u'bottom', u'panget', u'otherwise', u'yet', u'put', u'much', u'far', u'thank',  u'-PRON-', u'go', u'play',u'app', u'from', u'subject', u're', u'edu', u'use',  u'album', u'get', u'would', u'already',u'xe',u'vez', u'include', u'litt', u'sort', u'dayu', u'nice', u'dont', u'i', u'want', u'that'])
import warnings
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yubozhang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
with open('./android/ebay/total_info.txt') as f:
    reader = csv.reader(f, delimiter="\t")
    d = list(reader)

ratings = []
reviews = []
titles = []
dates = []
versions = []

for line in d:
    vals = line[0].split("******")
    ratings.append(float(vals[0]))
    reviews.append(vals[1])
    dates.append(vals[2])
    versions.append(vals[3])

In [7]:
len(reviews)

35483

In [8]:
reviews[0:10]

['new app be so slow . shop be no longer fun . i wish i could go back to last version . hate .',
 'i give my real email adress and i sit for a 1 2 hour try diffrent name .',
 'be have trouble with the app so i uninstalled and reinstall . the reinstall version have lose most of the info i use ? the only thing improve on this version be the pic .',
 "you do n't need an account to check thing out , but then you can convince your parent into make one and buy it .",
 "i use to have it on my tablet and i like it but something remove it and i do n't remove it now anytime i try to put it back on my tablet it wo n't install it download but it wo n't install i can not find anything wrong with my android installer i hope ebay read s this review and look into the problem the error message i get read s unknown error during install 24 .",
 'it good but there be no cash and cary option .',
 'i love it , it really cool and i make a ton of money . thank a million , ebay !',
 'so everything will be go s

In [9]:
def remove_stopwords(texts, stop_words):
    corp=[]
    for doc in texts:
        fix_doc=[]
        for word in doc:
            if word not in stop_words:
                fix_doc.append(word)
        corp.append(fix_doc)
    return corp
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        if sent != []:
            doc = nlp(" ".join(sent)) 

            texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
 
    return texts_out
def sent_to_words(sentences):
        for sentence in sentences:
            yield(gensim.utils.simple_preprocess(str([sentence]), deacc=True)) 

In [19]:
data_words = list(sent_to_words(reviews))

In [20]:
pyLDAvis.save_html(vis, 'lda.html')

In [11]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) 
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)





/Users/yubozhang/anaconda/lib/python2.7/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [12]:
nlp = spacy.load('en', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
data_words_nostops = remove_stopwords(data_lemmatized, stop_words)
data_words_bigrams = make_bigrams(data_words_nostops)

data_lemmatized = data_words_bigrams

In [17]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


warnings.filterwarnings("ignore",category=DeprecationWarning)

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)


In [18]:
warnings.filterwarnings("ignore",category=DeprecationWarning)
# Visualize the topics
score =1
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
9      20.065056        1       1  0.438273  0.078162
6      14.914978        1       2  0.030125 -0.436619
0      10.498384        1       3 -0.050878  0.033943
5       9.711154        1       4 -0.056575  0.056145
4       9.519139        1       5 -0.025382  0.043689
1       8.255690        1       6 -0.068712  0.039426
3       8.158638        1       7 -0.055304  0.043300
7       7.129101        1       8 -0.068380  0.050447
8       6.802588        1       9 -0.070374  0.050892
2       4.945275        1      10 -0.072793  0.040616, topic_info=     Category         Freq          Term        Total  loglift  logprob
term                                                                   
5077  Default  8871.000000          item  8871.000000  30.0000  30.0000
2795  Default  6951.000000        update  6951.000000  29.0000  29.0000
2324  Default  6259.000000          work  6259.000000  28.0000  28.0000
5800  Default  4961.000000          good  4961.000000  27.0000  27.0000
142   Default  5042.000000           fix  5042.000000  26.0000  26.0000
442   Default  4214.000000          easy  4214.000000  25.0000  25.0000
1748  Default  4073.000000         great  4073.000000  24.0000  24.0000
7983  Default  3001.000000       problem  3001.000000  23.0000  23.0000
634   Default  3252.000000          love  3252.000000  22.0000  22.0000
5697  Default  3974.000000        search  3974.000000  21.0000  21.0000
1463  Default  2373.000000    connection  2373.000000  20.0000  20.0000
2423  Default  2174.000000       network  2174.000000  19.0000  19.0000
5931  Default  2298.000000          find  2298.000000  18.0000  18.0000
7914  Default  3553.000000          list  3553.000000  17.0000  17.0000
3554  Default  1967.000000         issue  1967.000000  16.0000  16.0000
1502  Default  1932.000000         error  1932.000000  15.0000  15.0000
8113  Default  2284.000000        option  2284.000000  14.0000  14.0000
3750  Default  1388.000000          look  1388.000000  13.0000  13.0000
4954  Default  2087.000000  notification  2087.000000  12.0000  12.0000
5340  Default  1051.000000         track  1051.000000  11.0000  11.0000
3644  Default  1556.000000        mobile  1556.000000  10.0000  10.0000
4625  Default  1531.000000      purchase  1531.000000   9.0000   9.0000
5930  Default  1270.000000          fine  1270.000000   8.0000   8.0000
6114  Default  1504.000000          back  1504.000000   7.0000   7.0000
7328  Default  1138.000000       picture  1138.000000   6.0000   6.0000
8723  Default  1131.000000          view  1131.000000   5.0000   5.0000
109   Default  1473.000000       feature  1473.000000   4.0000   4.0000
7943  Default  1746.000000       version  1746.000000   3.0000   3.0000
6092  Default  1729.000000           bid  1729.000000   2.0000   2.0000
1204  Default   992.000000          send   992.000000   1.0000   1.0000
...       ...          ...           ...          ...      ...      ...
5340  Topic10  1050.191650         track  1051.150513   3.0058  -2.5995
874   Topic10   762.292480          slow   763.251343   3.0055  -2.9199
2672  Topic10   758.406799         photo   759.365723   3.0055  -2.9250
3318  Topic10   656.374146           log   657.333069   3.0053  -3.0695
2724  Topic10   457.143921          post   458.102844   3.0046  -3.4312
6476  Topic10   453.906097       payment   454.864960   3.0046  -3.4383
2097  Topic10   368.783813          less   369.742676   3.0041  -3.6460
2494  Topic10   347.100769    convenient   348.059662   3.0040  -3.7066
3692  Topic10   275.031555         close   275.990387   3.0033  -3.9394
1299  Topic10   213.221344     everytime   214.180206   3.0023  -4.1939
3977  Topic10   209.971542      download   210.930359   3.0022  -4.2093
5008  Topic10   205.800598       classic   206.759583   3.0021  -4.2293
6789  Topic10   200.214935         enjoy   201.173843   3.0020  -4.25

In [16]:
modeltopic = lda_model.print_topics()
tp = ['bug', 'great', 'speed','findissue','easy','error','average','problem','connect','search']

In [59]:
for i in range(len(modeltopic)):
    print 'topic: ', tp[i], ' -> ', modeltopic[i] 
    

topic:  bug  ->  (0, u'0.209*"work" + 0.168*"fix" + 0.042*"fine" + 0.039*"change" + 0.034*"bad" + 0.029*"crash" + 0.026*"tell" + 0.023*"stop" + 0.020*"screen" + 0.020*"first"')
topic:  great  ->  (1, u'0.210*"good" + 0.048*"picture" + 0.034*"pc" + 0.029*"description" + 0.026*"quick" + 0.024*"seem" + 0.023*"product" + 0.021*"user" + 0.015*"instead" + 0.014*"email"')
topic:  speed  ->  (2, u'0.074*"track" + 0.054*"slow" + 0.054*"photo" + 0.046*"log" + 0.032*"post" + 0.032*"payment" + 0.026*"less" + 0.025*"convenient" + 0.019*"close" + 0.015*"everytime"')
topic:  findissue  ->  (3, u'0.099*"find" + 0.084*"issue" + 0.059*"look" + 0.048*"view" + 0.044*"read" + 0.042*"something" + 0.033*"price" + 0.022*"upload" + 0.020*"awesome" + 0.020*"recent"')
topic:  easy  ->  (4, u'0.155*"easy" + 0.150*"great" + 0.120*"love" + 0.037*"ship" + 0.030*"well" + 0.028*"money" + 0.024*"order" + 0.022*"part" + 0.021*"fast" + 0.014*"navigate"')
topic:  error  ->  (5, u'0.250*"update" + 0.070*"error" + 0.032*"la